In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.chdir("/content/drive/MyDrive/colab_data/projects/nlp")

!pip install --no-deps --force-reinstall /content/drive/MyDrive/colab_data/misc_dist/krk_ml_utils-0.0.1-py3-none-any.whl

Processing /content/drive/MyDrive/colab_data/misc_dist/krk_ml_utils-0.0.1-py3-none-any.whl


In [3]:
import jax
import jax.numpy as jnp
import optax
from flax import nnx
from functools import partial

#
from krk_ml_utils.transformers import Vanilla_Transformer_v1
from krk_ml_utils.datasets import JaxNLPDataLoader, NumpyDataset

# --- 1. Model & Training Hyperparameters ---
VOCAB_SIZE = 32002
D_MODEL = 512
MAX_SEQ_LENGTH = 4000
#MAX_SEQ_LENGTH = 2048
NUM_LAYERS_ENC = 6
NUM_LAYERS_DEC = 6
NUM_HEADS_ENC = 8
NUM_HEADS_DEC = 8
D_FF_ENC = 2024
D_FF_DEC = 2024
DROPOUT_RATE = 0.1
#LEARNING_RATE = 1e-3
NUM_EPOCHS = 10
BATCH_SIZE = 8
PAD_TOKEN_ID = 32000 # Custom token added to tokenizer
SEED = 42

LABEL_SMOOTHING_ALPHA = 0.1

WARMUP_STEPS = 4000 # From the paper for the base model
#WARMUP_STEPS = 1500 # Smaller value for my dataset
ADAM_B1 = 0.9
ADAM_B2 = 0.98
ADAM_EPS = 1e-9

# --- 2. Instantiate Model, Optimizer, and Metrics ---
print("Initializing model components...")
model = Vanilla_Transformer_v1(
    vocab_size=VOCAB_SIZE, d_model=D_MODEL, max_seq_length=MAX_SEQ_LENGTH,
    num_layers_enc=NUM_LAYERS_ENC, num_layers_dec=NUM_LAYERS_DEC,
    num_heads_enc=NUM_HEADS_ENC, num_heads_dec=NUM_HEADS_DEC,
    d_dff_enc=D_FF_ENC, d_dff_dec=D_FF_DEC,
    seed=SEED, dropout_rate=DROPOUT_RATE
)

print(f"JAX sees the following devices: {jax.devices()}")

Initializing model components...
JAX sees the following devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


In [4]:
# Modify your loss function
def transformer_loss_fn_from_xy(model: nnx.Module, batch: tuple, pad_token_id: int = -1):
    source_tokens, full_target_sequence = batch
    decoder_input_tokens = full_target_sequence[:, :-1]
    labels = full_target_sequence[:, 1:]

    logits = model(
        source_tokens=source_tokens,
        target_tokens=decoder_input_tokens,
        training=True,
        pad_token_id=pad_token_id
    )

    vocab_size = logits.shape[-1]

    # --- MODIFIED LOSS CALCULATION ---
    # 1. Create smoothed, one-hot labels
    smoothed_labels_one_hot = optax.smooth_labels(
        jax.nn.one_hot(labels, num_classes=vocab_size),
        alpha=LABEL_SMOOTHING_ALPHA
    )

    # 2. Calculate cross entropy with the smoothed labels
    loss_values = optax.softmax_cross_entropy(logits, smoothed_labels_one_hot)

    # 3. Apply padding mask (loss is now per-token, not per-logit)
    padding_mask = (labels != pad_token_id)
    masked_loss = loss_values * padding_mask

    # Normalize by the number of non-padded tokens
    mean_loss = jnp.sum(masked_loss) / jnp.sum(padding_mask)

    return mean_loss, logits

In [5]:




# For a Transformer, the primary metric is perplexity, which is exp(cross_entropy_loss).
# So, just tracking the average loss is sufficient and the most important metric.
metrics = nnx.MultiMetric(
    loss=nnx.metrics.Average('loss'),
)

# --- 3. Create the Custom Loss Function ---
# We use functools.partial to "bake in" the pad_token_id.
# This makes the function signature match what train_flax_model expects.
loss_fn_with_padding = partial(transformer_loss_fn_from_xy, pad_token_id=PAD_TOKEN_ID)

In [6]:
from krk_ml_utils import datasets

# Create the Data Loader class with the correct padding token
JaxNLPDataLoader = datasets.create_jax_nlp_dataloader(
    pad_value=PAD_TOKEN_ID, max_len_targets=1157, max_len_features=2128
)

# --- 4. Load the Dataset ---
train_ds = datasets.NumpyDataset(
    file_path="./data/train.npz",
    features_key="x",
    labels_key="y",
    rngs=None,
    allow_pickle=True,
    preload=True
)

train_loader = JaxNLPDataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True
                                #,num_workers=8
                                #,prefetch_factor=8
                                )

In [7]:
# Load the test dataset
test_ds = datasets.NumpyDataset(
    file_path="./data/test.npz",
    features_key="x",
    labels_key="y",
    rngs=None,
    allow_pickle=True,
    preload=True
)
test_loader = JaxNLPDataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False
                               #,num_workers = 8
                               #,prefetch_factor=8
                               )

In [8]:
### Learning Schedule

# Create the custom learning rate schedule from the paper
lr_schedule = optax.warmup_cosine_decay_schedule(
    init_value=0.0,
    peak_value=D_MODEL**-0.5,
    warmup_steps=WARMUP_STEPS,
    # A very long decay is similar to the paper's inverse sqrt decay
    decay_steps=NUM_EPOCHS * len(train_loader),
    end_value=0.0
)

# A more direct implementation of the paper's schedule:
def paper_lr_schedule(step: int):
    step = jnp.maximum(step, 1) # prevent step=0
    arg1 = step**-0.5
    arg2 = step * (WARMUP_STEPS**-1.5)
    return (D_MODEL**-0.5) * jnp.minimum(arg1, arg2)

# Also use the paper's beta values.
optimizer = nnx.Optimizer(model, optax.adam(
    learning_rate=paper_lr_schedule, # Use the custom schedule
    b1=ADAM_B1,
    b2=ADAM_B2,
    eps=ADAM_EPS
))

In [9]:
%%javascript
let counter = 0;
setInterval(() => {
  console.log("Background JS running:", counter++);
  document.querySelector("colab-connect-button").click();
}, 30000);  // Clicks every 30 seconds

<IPython.core.display.Javascript object>

In [ ]:
from krk_ml_utils.training_multi_gpu import train_flax_lm
checkpoint_dir = "./transformer_checkpoints_multi_gpu_with_lr"
# --- 5. Start the Training Run ---
print("Starting training...")
trained_model, history = train_flax_lm(
    model=model,
    optimizer=optimizer,
    metrics=metrics,
    loss_fn=loss_fn_with_padding,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    num_epochs=NUM_EPOCHS,
    checkpoint_dir=checkpoint_dir,
    save_every_steps=2000,
    accumulation_steps=100,
    log_train_metrics_every_steps=500,
    eval_every_steps=3000,
    resume_from_checkpoint=True
)

print("Training finished!")

Starting training...
Using device mesh with 8 devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]
Gradient accumulator will be initialized on first batch.
Found latest checkpoint: ./transformer_checkpoints_multi_gpu_with_lr/epoch_0001_step_00138000
Error converting to JAX array: JAX array with PRNGKey dtype cannot be converted to a NumPy array. Use jax.random.key_data(arr) if you wish to extract the underlying integer array.
Dtype: key<fry>, Shape: (), Value: Array((), dtype=key<fry>) overlaying:
[ 0 42]
Erro